Imports


In [22]:
import pandas as pd
import os, sys
sys.path.append(os.path.abspath(".."))

from src.db_connect import engine


File Loading

In [13]:
file_name = "gdp.csv"
raw_path = os.path.join("..", "data", "raw", file_name)

In [14]:
df = pd.read_csv(raw_path, skiprows=4)
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,Gross capital formation (% of GDP),NE.GDI.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,20.575010,20.551182,22.401724,22.438504,24.178783,21.788181,20.917522,21.461002,NaN,NaN
1,Africa Eastern and Southern,AFE,Gross capital formation (% of GDP),NE.GDI.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,21.500533,20.933449,20.957207,21.538807,20.391104,20.124374,21.343422,19.800942,NaN,NaN
2,Afghanistan,AFG,Gross capital formation (% of GDP),NE.GDI.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,11.500000,13.000000,16.700000,15.200000,NaN,NaN
3,Africa Western and Central,AFW,Gross capital formation (% of GDP),NE.GDI.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AGO,Gross capital formation (% of GDP),NE.GDI.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,27.214709,24.130306,17.869421,17.712255,26.490400,26.437362,26.256018,23.793250,NaN,NaN


Data cleaning: remove unnecessary columns and reshape dataframe

In [17]:
# Remove unnecessary columns
df = df.drop(columns=["Indicator Name", "Indicator Code"])
df.head()

,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.575010,20.551182,22.401724,22.438504,24.178783,21.788181,20.917522,21.461002,NaN,NaN
1,Africa Eastern and Southern,AFE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.500533,20.933449,20.957207,21.538807,20.391104,20.124374,21.343422,19.800942,NaN,NaN
2,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,11.500000,13.000000,16.700000,15.200000,NaN,NaN
3,Africa Western and Central,AFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.214709,24.130306,17.869421,17.712255,26.490400,26.437362,26.256018,23.793250,NaN,NaN


Saving clean data to a file

In [18]:
df_long = df.melt(
    id_vars=["Country Name", "Country Code"],
    var_name="Year",
    value_name="Value"
)
df_long.head()

,Country Name,Country Code,Year,Value
0,Aruba,ABW,1960,NaN
1,Africa Eastern and Southern,AFE,1960,NaN
2,Afghanistan,AFG,1960,NaN
3,Africa Western and Central,AFW,1960,NaN
4,Angola,AGO,1960,NaN


Connecting do database 

In [19]:
df_long = df_long.dropna(subset=["Value"])
df_long["Year"] = df_long["Year"].astype(int)
df_long.head()

,Country Name,Country Code,Year,Value
9,Argentina,ARG,1960,23.479234
13,Australia,AUS,1960,31.569760
16,Burundi,BDI,1960,6.250000
18,Benin,BEN,1960,11.424972
19,Burkina Faso,BFA,1960,8.519939


In [21]:
clean_path = os.path.join("..", "data", "clean", file_name)
df_long.to_csv(clean_path, index=False)

In [23]:
table_name = "gdp_data"
df_long.to_sql(table_name, engine, if_exists="replace", index=False)

584

In [26]:
df_check = pd.read_sql("SELECT * FROM gdp_data LIMIT 10;", engine)
df_check

,Country Name,Country Code,Year,Value
0,Argentina,ARG,1960,23.479234
1,Australia,AUS,1960,31.569760
2,Burundi,BDI,1960,6.250000
3,Benin,BEN,1960,11.424972
4,Burkina Faso,BFA,1960,8.519939
5,Bangladesh,BGD,1960,6.894266
6,Brazil,BRA,1960,17.523227
7,Botswana,BWA,1960,6.015738
8,Central African Republic,CAF,1960,19.636356
9,Chile,CHL,1960,14.076841
